<a href="https://colab.research.google.com/github/AnnaHauschild/Artificial-Intelligence/blob/main/Trustworthy_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    import secml
    import foolbox
    import sklearn
except Exception:
    %pip install 'secml[pytorch]' foolbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.0/464.0 kB 13.9 MB/s eta 0:00:00


# Exercise 1

Complete the following tasks using SecML

Create an SVM with scikit-learn (use SVC, leave the default parameters) and wrap it with the scikit-learn SecML wrapper seen during the course.

In [ ]:
from sklearn.svm import SVC
from secml.ml.classifiers import CClassifierSkLearn

# complete here
model = SVC()
secml_model = CClassifierSkLearn(model)

2024-11-12 10:27:09,876 - secml.settings - INFO - New `SECML_HOME_DIR` created: /root/secml-data
2024-11-12 10:27:09,876 - secml.settings - INFO - New `SECML_HOME_DIR` created: /root/secml-data


INFO:secml.settings:New `SECML_HOME_DIR` created: /root/secml-data


2024-11-12 10:27:09,899 - secml.settings - INFO - Default configuration file copied to: /root/secml-data/secml.conf
2024-11-12 10:27:09,899 - secml.settings - INFO - Default configuration file copied to: /root/secml-data/secml.conf


INFO:secml.settings:Default configuration file copied to: /root/secml-data/secml.conf


2024-11-12 10:27:09,908 - secml.settings - INFO - New `SECML_DS_DIR` created: /root/secml-data/datasets
2024-11-12 10:27:09,908 - secml.settings - INFO - New `SECML_DS_DIR` created: /root/secml-data/datasets


...

INFO:secml.settings:New `SECML_EXP_DIR` created: /root/secml-data/experiments


2024-11-12 10:27:09,951 - secml.settings - INFO - New `SECML_LOGS_DIR` created: /root/secml-data/logs
2024-11-12 10:27:09,951 - secml.settings - INFO - New `SECML_LOGS_DIR` created: /root/secml-data/logs


INFO:secml.settings:New `SECML_LOGS_DIR` created: /root/secml-data/logs


2024-11-12 10:27:09,970 - secml.settings - INFO - New `SECML_PYTORCH_DIR` created: /root/secml-data/pytorch-data
2024-11-12 10:27:09,970 - secml.settings - INFO - New `SECML_PYTORCH_DIR` created: /root/secml-data/pytorch-data


INFO:secml.settings:New `SECML_PYTORCH_DIR` created: /root/secml-data/pytorch-data


Create a dataset consisting of 500 samples of 2D points from two clusters (CDLRandomBlobs), whose centers are located in (-2, 2) and (2,2) with standard deviation = 0.5. Normalize the dataset between 0 and 1 (CNormalizerMinMax).

In [ ]:
# complete here
random_state = 999

n_features = 2   # number of features
n_samples = 500   # number of samples
centers = [[-2, 2], [2, 2]]   # centers of the clusters
cluster_std = 0.5   # standard deviation of the clusters

from secml.data.loader import CDLRandomBlobs
dataset = CDLRandomBlobs(n_features=n_features,
                         centers=centers,
                         cluster_std=cluster_std,
                         n_samples=n_samples,
                         random_state=random_state).load()

# normalize the data
from secml.ml.features import CNormalizerMinMax
nmz = CNormalizerMinMax()
dataset.X = nmz.fit_transform(dataset.X)


Divide the data with the SecML splitter (CTrainTestSplit), using 400 samples per il training e 100 per il test.

In [ ]:
# complete here
n_tr = 400  # number of training set samples
n_ts = 100  # number of test set samples

# split in training and test
from secml.data.splitter import CTrainTestSplit
splitter = CTrainTestSplit(
    train_size=n_tr, test_size=n_ts, random_state=random_state)
tr, ts = splitter.split(dataset)



Fit the classifier on the training data.

In [ ]:
# complete here
secml_model.fit(tr.X,tr.Y)

Compute the accuracy on the test set using the CMetricAccuracy object.

In [ ]:
# complete here
predictions = secml_model.predict(ts.X)

# this class computes the accuracy of a model
from secml.ml.peval.metrics import CMetricAccuracy
metric = CMetricAccuracy()

accuracy = metric.performance_score(y_true=ts.Y, y_pred=predictions)

print("Accuracy on the test set", accuracy)

Plot the learned decision function using the ad-hoc method of the CFigure class.

In [ ]:
# complete here
fig = CFigure(width=5, height=5)

# convenience function for plotting the decision function of a classifier
fig.sp.plot_decision_regions(secml_model, n_grid_points=200)

fig.sp.plot_ds(ts)
fig.sp.grid(grid_on=False)

fig.sp.title("Classification regions")
fig.show()

# Exercise 2

We provide you the code for training a simple model on 2D data. Execute the cell to create and fit the model.

In [ ]:
%matplotlib inline
import torch
torch.manual_seed(111)
from torch.nn.functional import relu, softmax
from torch.optim import SGD
from secml.ml.classifiers import CClassifierPyTorch
from secml.ml.features import CNormalizerMinMax
from secml.data.splitter import CTrainTestSplit
from secml.data.loader import CDLRandomBlobs
from secml.ml.features import CNormalizerMinMax
from secml.figure import CFigure
from secml.ml.peval.metrics import CMetricAccuracy

class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = torch.nn.Linear(2, 50)
        self.lin2 = torch.nn.Linear(50, 50)
        self.lin3 = torch.nn.Linear(50, 3)

    def forward(self, x):
        x = relu(self.lin1(x))
        x = relu(self.lin2(x))
        y = self.lin3(x)
        return y

net = Net()
optimizer = SGD(net.parameters(), lr=0.01, momentum=0.9)
loss_function = torch.nn.CrossEntropyLoss()
from secml.ml.classifiers import CClassifierPyTorch
clf = CClassifierPyTorch(model=net,
                         loss=loss_function,
                         optimizer=optimizer,
                         epochs=30,
                         batch_size=32,
                         random_state=31)


dataset = CDLRandomBlobs(n_features=2,
                         centers=[[0,0], [1,1], [0,1]],
                         cluster_std=0.2,
                         n_samples=500,
                         random_state=111).load()

dataset.X = CNormalizerMinMax().fit_transform(dataset.X)
splitter = CTrainTestSplit(train_size=475, test_size=25, random_state=8)
tr, ts = splitter.split(dataset)

clf.fit(tr.X, tr.Y)

print(f"Accuracy is {CMetricAccuracy().performance_score(clf.predict(ts.X), ts.Y)}")
fig = CFigure()
fig.sp.plot_ds(ts)
fig.sp.plot_decision_regions(clf, n_grid_points=50)

Compute a security evaluation curve of the trained model using the corresponding SecML object (CSecEval) as shown in class, with the following setup.

* Use the CFoolboxPGDLInf attack from the SecML library, with 5 iterations and 0.05 as absolute step size, no random start, lower bound 0 and upper bound 1;

* Use at least 5 values for the perturbation budget epsilon.
Since it is an L-inf attack and the domain is bounded between 0 and 1, which is the maximum perturbation norm that the attack can reach?



Answer: Since it is an L-inf attack. the maximum value of epsilon can become 1. Assuming the sample lays in (0,0) then the maximum pertubation can be at (1,1) without escaping from input domain. L_inf = max(x). If it is not normed the maximum would be k.

In [ ]:
# complete here
from secml.array import CArray
from secml.adv.attacks.evasion import CFoolboxPGDLinf
from secml.adv.seceval import CSecEval

n_retrains = 5
how_many_adv = 200
y_target = None
lb, ub = 0, 1
eps = 0.2
alpha = 0.05
steps = 10

# perturbation levels to test
epsilon_vals = CArray.linspace(start=0, stop=1, num=5)

pgd_attack = CFoolboxPGDLinf(clf, y_target,
                         lb=lb, ub=ub,
                         epsilons=eps,
                         abs_stepsize=alpha,
                         steps=steps,
                         random_start=False)
sec_eval = CSecEval(attack=pgd_attack, param_name='epsilon', param_values=epsilon_vals)
sec_eval.run_sec_eval(ts)


Plot the security evaluation curve using the ad-hoc method of the CFigure class.

In [ ]:
# complete here

%matplotlib inline
from secml.figure import CFigure

fig = CFigure(height=5, width=10)

# convenience function for plotting the Security Evaluation Curve
fig.sp.plot_sec_eval(sec_eval.sec_eval_data, marker='o', label='regular', show_average=True)
fig.show()

# Exercise 3

We provide the code for an example implementation of gradient descent attack.
The implementation contains a bug: find it, fix it, and motivate your choice.

In [ ]:
from secml.array import CArray
from secml.ml import CClassifier
from secml.ml.classifiers.loss import CLossCrossEntropy


def pgd_l2_targeted(clf : CClassifier, x : CArray, y_original : CArray, y_target : int, steps : int=20, eps : float=0.5, step_size : float=0.05, lb : int = 0, ub : int = 1):
    loss_func = CLossCrossEntropy()
    x_adv = x.deepcopy()
    for i in range(steps):
        scores = clf.decision_function(x_adv)
        loss_gradient = loss_func.dloss(y_true=y_original, score=scores, pos_label=y_target)
        clf_gradient = clf.grad_f_x(x_adv, y_target)
        gradient = clf_gradient * loss_gradient

        if gradient.norm() != 0:
            gradient /= gradient.norm()

      #bug: the + is supposed to be a -. Since it is a targeted attack we want to go closer to the center of the target class
      #with + int the untargeted attack we go far away from the original class
        x_adv = x_adv - gradient
        delta = x_adv - x
        if delta.norm() > eps:
            delta = delta / delta.norm()
            x_adv = x + delta * eps
        x_adv = x_adv.clip(lb, ub)

    scores = clf.decision_function(x_adv)
    print(f'Adv label: {scores.argmax()}')
    return x_adv